In [341]:
from datetime import date
from prophet import Prophet
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import yfinance as yf

In [342]:
df = pd.read_csv("../Resources/btcjoin.csv", parse_dates=['date'])
btc_df = yf.download('BTC-USD')
btc_df = btc_df.reset_index()
btc_df = btc_df.loc[(btc_df['Date'] > '2022-10-25')]
btc_df['Close']=btc_df['Close'].astype("float")
df['price']=df['price'].str.replace(',','')
df['price']=df['price'].astype("float")
btc_df = btc_df.rename(columns={"Close": "price", "Date":"date"})
df = pd.merge(df, btc_df, on=['date', 'price'], how='outer')
df = df.rename(columns={"value": "wallets"})
df = df.drop(columns=['volume','change', 'low', 'high', 'open','Open','High','Low','Adj Close', 'Volume', 'Unnamed: 0', "wallets", "address", "mined"])
df['200D'] = df['price'].rolling(200).mean()
df['300D'] = df['price'].rolling(300).mean()
df['50D'] = df['price'].rolling(50).mean()
df = df.dropna()
df['meanavge'] = (df['200D'] + df['300D'] + df['50D'] )/3
df = df.drop(columns=['200D','300D', '50D'])
df['meanvalue'] = df["price"] - df["meanavge"]
df['status'] = df['meanvalue'].apply(lambda x: '1' if x > 0 else '0')
df['status']=df['status'].astype("object")
df['price-meanavge']=df['price'] - df['meanavge']
df['move%'] = df['price-meanavge']/(df['price'] + df['meanavge'])
bins = [-0.43, -0.1, 0, 0.1, 0.43]
group_names = ["Severely Oversold","Oversold", "Neutral","Overbought"]
df["Valuation"] = pd.cut(df["move%"], bins, labels=group_names)
pricefrommean = df.meanvalue.iloc[-1].round(2)
currentzone = df.Valuation.iloc[-1]
delta1cycle = df.index[df['date']=='2015-09-15'].tolist()[0] - df.index[df['date']=='2013-12-04'].tolist()[0]
delta2cycle = df.index[df['date']=='2019-04-2'].tolist()[0] - df.index[df['date']=='2017-12-17'].tolist()[0] 
averageunder = int((delta1cycle+delta2cycle)/2)
delta1fromp2p = df.index[df['date']=='2017-03-17'].tolist()[0] - df.index[df['date']=='2013-12-04'].tolist()[0]
delta2fromp2p = df.index[df['date']=='2020-11-29'].tolist()[0] - df.index[df['date']=='2017-12-17'].tolist()[0] 
averagep2p = int((delta1fromp2p+delta2fromp2p)/2)
sincealltimehigh = df.index[-1] - df.index[df['price']==df.price.max()].tolist()[0]

[*********************100%***********************]  1 of 1 completed


In [343]:
import plotly.graph_objects as go

fig = go.Figure(go.Indicator(
    domain = {'x': [0, 1], 'y': [0, 1]},
    value = df.price.iloc[-1],
    mode = "gauge+number+delta",
    title = {'text': "Current Price and ATH Delta"},
    delta = {'reference': df.price[df['price']==df.price.max()].tolist()[0]},
    gauge = {'axis': {'range': [None, df.price[df['price']==df.price.max()].tolist()[0]]},
    'bar': {'color': "orange"}}))

fig.show()
fig.write_html("../static/priceath.html")

In [344]:
df.price.iloc[-1]

18869.587890625

In [345]:
import plotly.graph_objects as go

fig = go.Figure(go.Indicator(
    domain = {'x': [0, 1], 'y': [0, 1]},
    value = df.meanavge.iloc[-1],
    mode = "number",
    title = {'text': "Current Mean Average"},
    gauge = {'axis': {'range': [None, df.price[df['price']==df.price.max()].tolist()[0]]},
    'bar': {'color': "orange"}}))

    

fig.show()
fig.write_html("../static/meanaverage.html")

In [346]:
import plotly.graph_objects as go

fig = go.Figure(go.Indicator(
    domain = {'x': [0, 1], 'y': [0, 1]},
    value =  df.price[df['price']==df.price.max()].tolist()[0],
    mode = "number",
    title = {'text': "Previous ATH"},
    gauge = {'axis': {'range': [None, df.price[df['price']==df.price.max()].tolist()[0]]},
    'bar': {'color': "orange"}}))

    

fig.show()
fig.write_html("../static/previousATH.html")

In [347]:
import plotly.graph_objects as go

fig = go.Figure(go.Indicator(
    domain = {'x': [0, 1], 'y': [0, 1]},
    value = df.price.iloc[-1] - df.meanavge.iloc[-1],
    mode = "gauge+number",
    title = {'text': "Current Price Meanaverage Delta"},
    delta = {'reference': 0},
    gauge = {'axis': {'range': [None, df.meanvalue[df['meanvalue']==df.meanvalue.min()].tolist()[0]]},
    'bar': {'color': "orange"}}))

fig.show()
fig.write_html("../static/price-mean.html")

In [363]:
date = '2023-01-01'



bitcoin =yf.Ticker('BTC-USD').history(period='ytd',interval='1d',actions=False).reset_index()
bitcoin = format((((bitcoin.Close.iloc[-1]/bitcoin.Close.iloc[0])-1)))

spy500 = yf.download('SPY').reset_index()
spy500 = spy500.loc[(spy500['Date'] >= date)]
spy500 = format((((spy500.Close.iloc[-1]/spy500.Close.iloc[0])-1)))

Gold = yf.download('GC=F').reset_index()
Gold = Gold.loc[(Gold['Date'] >= date)]
Gold = format((((Gold.Close.iloc[-1]/Gold.Close.iloc[0])-1)))

nas = yf.download('^IXIC').reset_index()
nas = nas.loc[(nas['Date'] >= date)]
nas = format((((nas.Close.iloc[-1]/nas.Close.iloc[0])-1)))

DJ = yf.download('^DJI').reset_index()
DJ = DJ.loc[(DJ['Date'] >= date)]
DJ = format((((DJ.Close.iloc[-1]/DJ.Close.iloc[0])-1)))

Oil = yf.download('CL=F').reset_index()
Oil = Oil.loc[(Oil['Date'] >= date)]
Oil = format((((Oil.Close.iloc[-1]/Oil.Close.iloc[0])-1)))

Rus = yf.download('^RUT').reset_index()
Rus = Rus.loc[(Rus['Date'] >= date)]
Rus = format((((Rus.Close.iloc[-1]/Rus.Close.iloc[0])-1)))

TLT = yf.download('TLT').reset_index()
TLT = TLT.loc[(TLT['Date'] >= date)]
TLT = format((((TLT.Close.iloc[-1]/TLT.Close.iloc[0])-1)))

DXY = yf.download('DX=F').reset_index()
DXY = DXY.loc[(DXY['Date'] >= date)]
DXY = format((((DXY.Close.iloc[-1]/DXY.Close.iloc[0])-1)))


data = [["Bitcoin", bitcoin], ["Gold", Gold], ["Oil", Oil],["20yr Treasury", TLT], ["DXY", DXY], ["Russel 200", Rus], ["S&P 500", spy500], ["NASDAQ", nas], ["Dow Jones", DJ]]

# Create the pandas DataFrame
df = pd.DataFrame(data, columns=['Asset', 'YTD %'])
df['YTD %'] = df['YTD %'].astype(float).round(3) * 100

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [364]:
import plotly.express as px
fig = px.bar(df, x='Asset', y='YTD %', orientation='v', color="YTD %",color_continuous_scale=px.colors.sequential.Agsunset)
fig.layout.yaxis.tickformat = ',.%'
fig.update_layout(title_text='Year To Date Returns')
fig.update_layout(yaxis=dict(showticklabels=True))
fig.show()

In [361]:
import yfinance as yf
import pendulum
import matplotlib.pyplot as plt
price_history = yf.Ticker('TSLA').history(period='ytd',interval='1d',actions=False)
                                   
price_history

,Open,High,Low,Close,Volume
Date,,,,,
2023-01-03,118.470001,118.800003,104.639999,108.099998,231402800
2023-01-04,109.110001,114.589996,107.519997,113.639999,180389000
2023-01-05,110.510002,111.750000,107.160004,110.339996,157986300
2023-01-06,103.000000,114.389999,101.809998,113.059998,220575900
2023-01-09,118.959999,123.519997,117.110001,119.769997,190284000
2023-01-10,121.070000,122.760002,114.919998,118.849998,167642500
2023-01-11,122.089996,125.949997,120.510002,123.220001,183810800
2023-01-12,122.559998,124.129997,117.000000,123.559998,169089400
2023-01-13,116.550003,121.650002,115.599998,119.754997,106345894
